# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("../WeatherPy/output_data/cities.csv")
cities_df = pd.DataFrame(cities)
cities_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Temperature (F)
0,Bluff,-46.6000,168.3333,273.60,87,9,4.06,NZ,1624125440,33.08
1,Kapaa,22.0752,-159.3190,300.14,80,24,0.89,US,1624125440,80.85
2,Havøysund,70.9963,24.6622,280.64,89,98,1.94,NO,1624125441,45.75
3,Tuktoyaktuk,69.4541,-133.0374,289.15,51,40,5.14,CA,1624125441,61.07
4,Dharampur,20.5333,73.1833,299.61,81,95,4.98,IN,1624125441,79.90
...,...,...,...,...,...,...,...,...,...,...
524,Fort Nelson,58.8053,-122.7002,294.48,37,56,1.68,CA,1624125603,70.66
525,Kutum,14.2000,24.6667,303.32,18,27,4.05,SD,1624125637,86.58
526,Nabire,-3.3667,135.4833,296.87,93,99,1.50,ID,1624125637,74.97
527,Breţcu,46.0500,26.3000,290.02,99,96,1.88,RO,1624125638,62.64


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#storing locaiton and humidity values for a heat map
location_df = cities_df[["Lat", "Lng"]]
humidity_df = cities_df["Humidity"]

In [4]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [5]:
figure_layout = {
    'width': '9000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,center=(46.0, -5.0), zoom_level=2)
heatmap = gmaps.heatmap_layer(location_df, weights=humidity_df, point_radius = 15)
fig.add_layer(heatmap)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

A max temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.
Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [6]:
max_temp = 80
min_temp = 70
wind_speed = 10
cloud = 0

cities_filtered_df = cities_df.loc[(cities_df["Temperature (F)"] > min_temp) & (cities_df["Temperature (F)"] < max_temp) & (cities_df["Wind Speed"] < wind_speed) & (cities_df["Cloudiness"] == cloud),: ]
cities_filtered_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Temperature (F)
59,Āstāneh-ye Ashrafīyeh,37.2596,49.9444,296.95,71,0,0.75,IR,1624125459,75.11
126,Arequipa,-16.3989,-71.5350,294.19,21,0,4.12,PE,1624125483,70.14
170,Dondo,-19.6094,34.7431,295.88,84,0,2.35,MZ,1624125498,73.18
214,Benghazi,32.1167,20.0667,296.59,76,0,6.04,LY,1624125514,74.46
274,Pishin,30.5803,66.9961,298.96,7,0,0.85,PK,1624125534,78.73
291,Al Bardīyah,31.7561,25.0865,296.29,71,0,7.91,LY,1624125539,73.92
330,Ürümqi,43.8010,87.6005,296.68,33,0,4.00,CN,1624125376,74.62
334,Marsá Maţrūḩ,31.3525,27.2453,297.26,64,0,4.12,EG,1624125553,75.67


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = cities_filtered_df[['Lat','Lng',"City","Country"]]
hotel_df['Hotel Name'] = ""
hotel_df

/opt/anaconda3/envs/AKpython/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Lat,Lng,City,Country,Hotel Name
59,37.2596,49.9444,Āstāneh-ye Ashrafīyeh,IR,
126,-16.3989,-71.5350,Arequipa,PE,
170,-19.6094,34.7431,Dondo,MZ,
214,32.1167,20.0667,Benghazi,LY,
274,30.5803,66.9961,Pishin,PK,
291,31.7561,25.0865,Al Bardīyah,LY,
330,43.8010,87.6005,Ürümqi,CN,
334,31.3525,27.2453,Marsá Maţrūḩ,EG,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {    
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

for index,row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    #get locations from hotel_df
    params['location'] = f"{lat},{lng}"
    
    #add location to params
    #params['location'] = location

    response = requests.get(base_url, params=params).json()
    
    # extract results
    
    #print(response['results'][0]['name'])
    
    hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    
hotel_df


/opt/anaconda3/envs/AKpython/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Lat,Lng,City,Country,Hotel Name
59,37.2596,49.9444,Āstāneh-ye Ashrafīyeh,IR,Hotel Farrghi
126,-16.3989,-71.5350,Arequipa,PE,Casa de Avila Hotel
170,-19.6094,34.7431,Dondo,MZ,Arco Iris
214,32.1167,20.0667,Benghazi,LY,Tibesti Hotel
274,30.5803,66.9961,Pishin,PK,City Hotel Pishin
291,31.7561,25.0865,Al Bardīyah,LY,بورتو بردية
330,43.8010,87.6005,Ürümqi,CN,Hua Ling Grand Hotel Urumqi
334,31.3525,27.2453,Marsá Maţrūḩ,EG,Beau Site


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}



fig = gmaps.figure(layout=figure_layout,center=(46.0, -5.0), zoom_level=2)


#print(fig)
#heatmap = gmaps.heatmap_layer(location_df, weights=humidity_df, point_radius = 15)
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(heatmap)
fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…